In [80]:
import pandas as pd

from tpot import TPOTRegressor
from dask.distributed import Client
import dask

# Import data

In [81]:
df = pd.read_csv('../Final/new_finish_data_3.csv')

In [82]:
df

,cc_week_1,cc_week_2,cc_week_3,cc_week_4,cc_week_5,cc_week_6,cc_week_7,cc_week_8,cc_week_9,cc_week_10,...,ocp_11,ocp_12,ocp_13,age_2,age_3,age_4,age_5,age_6,gender_1,gender_2
0,0.0,0.0,4700.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,1
1,1600.0,0.0,0.0,3500.0,2600.0,800.0,0.0,3800.0,1200.0,800.0,...,0,0,0,0,0,1,0,0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,1
4,0.0,0.0,5000.0,2000.0,0.0,5000.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,1
64996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,1
64997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0
64998,0.0,900.0,0.0,1000.0,0.0,0.0,0.0,2400.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,1


In [83]:
train = pd.read_csv('../data_pack/train.csv')
train.tail()

,id,income
49995,49996,48000
49996,49997,37000
49997,49998,19000
49998,49999,13000
49999,50000,21000


In [84]:
X = df[0:50000]
y = train.income

# TPOD Auto model.

In [85]:
client = Client()
client

C:\Users\INDY\Miniconda3\lib\site-packages\distributed\dashboard\core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:62850 Dashboard: http://127.0.0.1:62853/status,Cluster Workers: 4 Cores: 16 Memory: 17.09 GB


In [88]:
tpot = TPOTRegressor(generations=5, n_jobs=-1, use_dask=False, random_state=0, verbosity=2)

In [89]:
tpot.fit(X, y)

Generation 1 - Current best internal CV score: -2065796808.2556374
Generation 2 - Current best internal CV score: -2065796808.2556374
Generation 3 - Current best internal CV score: -2058198387.9048076
Generation 4 - Current best internal CV score: -2049949495.642366
Generation 5 - Current best internal CV score: -2047541041.5423064

Best pipeline: RandomForestRegressor(input_matrix, bootstrap=False, max_features=0.2, min_samples_leaf=8, min_samples_split=11, n_estimators=100)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
              disable_update_check=False, early_stop=None, generations=5,
              max_eval_time_mins=5, max_time_mins=None, memory=None,
              mutation_rate=0.9, n_jobs=-1, offspring_size=None,
              periodic_checkpoint_folder=None, population_size=100,
              random_state=0, scoring=None, subsample=1.0, template=None,
              use_dask=False, verbosity=2, warm_start=False)

In [27]:
tpot.export('model.py')

In [91]:
test = pd.read_csv('../data_pack/test.csv')
test

,id
0,50001
1,50002
2,50003
3,50004
4,50005
...,...
14995,64996
14996,64997
14997,64998
14998,64999


In [92]:
test['income'] = tpot.predict(df[50000:])

In [93]:
test.head()

,id,income
0,50001,29894.204685
1,50002,28211.292541
2,50003,38511.597125
3,50004,27258.647788
4,50005,42182.051643


In [94]:
test.to_csv('TPOT_NEW.csv', index=False)